In [1]:
import cobra
import pandas as pd

### Testing validity of cutsets on cancer model

In [2]:
func = lambda x: x.startswith('mcs_rsub') and not x.endswith('tgt')

In [3]:
dcsv = pd.read_csv("../results/output_subset_local_13.csv")

In [4]:
dcsv

,Unnamed: 0,mcs_m00003c,mcs_m00003s,mcs_m00004c,mcs_m00006c,mcs_m00007c,mcs_m00007m,mcs_m00008c,mcs_m00008s,mcs_m00009c,...,mcs_rsub_991,mcs_rsub_992,mcs_rsub_993,mcs_rsub_994,mcs_rsub_995,mcs_rsub_996,mcs_rsub_997,mcs_rsub_998,mcs_rsub_999,mcs_temp001x
0,0,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.13868,0.13868,0.13868,0.0,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.00000,0.00000,0.00000,0.0,-0.00150,-0.00150,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,0.00000,0.00000,-0.00075,0.0,-0.00075,-0.00075,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,0.00000,0.00000,-0.00075,0.0,-0.00075,-0.00075,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,0.00000,0.00000,-0.00075,0.0,-0.00075,-0.00075,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
lsup = []
sup = []
for k, csv in (dcsv > 0).iterrows():
    support = (list(csv.loc[csv].index))
    support = list(filter(func, support))
    print(support, len(support))
    lsup.append(len(support))
    sup.append(support)

['mcs_rsub_1101', 'mcs_rsub_1143', 'mcs_rsub_1173', 'mcs_rsub_1238', 'mcs_rsub_1628', 'mcs_rsub_1692', 'mcs_rsub_586', 'mcs_rsub_804'] 8
['mcs_rsub_169', 'mcs_rsub_439', 'mcs_rsub_625', 'mcs_rsub_75'] 4
['mcs_rsub_750', 'mcs_rsub_751', 'mcs_rsub_752'] 3
['mcs_rsub_750', 'mcs_rsub_751', 'mcs_rsub_752', 'mcs_rsub_820'] 4
['mcs_rsub_547', 'mcs_rsub_75'] 2
['mcs_rsub_1102'] 1
['mcs_rsub_418', 'mcs_rsub_443', 'mcs_rsub_445', 'mcs_rsub_488', 'mcs_rsub_547'] 5
['mcs_rsub_418', 'mcs_rsub_436', 'mcs_rsub_443', 'mcs_rsub_547'] 4
['mcs_rsub_418', 'mcs_rsub_436', 'mcs_rsub_444', 'mcs_rsub_483', 'mcs_rsub_547'] 5


In [6]:
model = cobra.io.read_sbml_model('../HepG2_medium.xml')
compressed_model = cobra.io.read_sbml_model("../new_reduced.xml")

Adding exchange reaction EX_ext_1 with default bounds for boundary metabolite: ext_1.
Adding exchange reaction EX_ext_10 with default bounds for boundary metabolite: ext_10.
Adding exchange reaction EX_ext_11 with default bounds for boundary metabolite: ext_11.
Adding exchange reaction EX_ext_12 with default bounds for boundary metabolite: ext_12.
Adding exchange reaction EX_ext_13 with default bounds for boundary metabolite: ext_13.
Adding exchange reaction EX_ext_14 with default bounds for boundary metabolite: ext_14.
Adding exchange reaction EX_ext_15 with default bounds for boundary metabolite: ext_15.
Adding exchange reaction EX_ext_16 with default bounds for boundary metabolite: ext_16.
Adding exchange reaction EX_ext_17 with default bounds for boundary metabolite: ext_17.
Adding exchange reaction EX_ext_18 with default bounds for boundary metabolite: ext_18.
Adding exchange reaction EX_ext_19 with default bounds for boundary metabolite: ext_19.
Adding exchange reaction EX_ext_2 

### Testing on compressed model

In [7]:
compressed_model.objective = "rsub_2116"

In [8]:
txtfile='../new_reactionSubsets.txt'
with open(txtfile, 'r') as f:
    dicto = eval(f.read())

In [9]:
from itertools import combinations

In [13]:
i = 1
for cs in sup:
    print(f"\nCUTSET N° {i} : \n\n")
    with compressed_model as m:
        for reac in cs:
            m.reactions.get_by_id(reac.strip("mcs_")).knock_out()
        opt = m.optimize()
    print(opt, cs)
    if len(cs) > 1 :
        for nm1 in combinations(cs, len(cs)-1):
            with compressed_model as m:
                for reac in nm1:
                    m.reactions.get_by_id(reac.strip("mcs_")).knock_out()
                opt = m.optimize()
            print('N-1', opt, nm1, 'not MIN cutset' if opt.objective_value < 1e-6 else '')
    i+=1


CUTSET N° 1 : 


<Solution 0.000 at 0x7f4601b1c220> ['mcs_rsub_1101', 'mcs_rsub_1143', 'mcs_rsub_1173', 'mcs_rsub_1238', 'mcs_rsub_1628', 'mcs_rsub_1692', 'mcs_rsub_586', 'mcs_rsub_804']
N-1 <Solution 0.000 at 0x7f4601b20ca0> ('mcs_rsub_1101', 'mcs_rsub_1143', 'mcs_rsub_1173', 'mcs_rsub_1238', 'mcs_rsub_1628', 'mcs_rsub_1692', 'mcs_rsub_586') not MIN cutset
N-1 <Solution 22.197 at 0x7f4601b1f5b0> ('mcs_rsub_1101', 'mcs_rsub_1143', 'mcs_rsub_1173', 'mcs_rsub_1238', 'mcs_rsub_1628', 'mcs_rsub_1692', 'mcs_rsub_804') 
N-1 <Solution 0.000 at 0x7f4601b20d30> ('mcs_rsub_1101', 'mcs_rsub_1143', 'mcs_rsub_1173', 'mcs_rsub_1238', 'mcs_rsub_1628', 'mcs_rsub_586', 'mcs_rsub_804') not MIN cutset
N-1 <Solution 0.000 at 0x7f4601b1f5b0> ('mcs_rsub_1101', 'mcs_rsub_1143', 'mcs_rsub_1173', 'mcs_rsub_1238', 'mcs_rsub_1692', 'mcs_rsub_586', 'mcs_rsub_804') not MIN cutset
N-1 <Solution 0.000 at 0x7f4601b231c0> ('mcs_rsub_1101', 'mcs_rsub_1143', 'mcs_rsub_1173', 'mcs_rsub_1628', 'mcs_rsub_1692', 'mcs_rsub_

### 